In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head( )

EPSG:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [3]:
sidewalks = gpd.read_file('../data/Sidewalks.geojson')
print(sidewalks.crs)
sidewalks.head( )

EPSG:4326


,evnt_lat,obj_pot,side,measure,curbtype,evnt_lon,sw_mat_ty,drv_ld,cr_slp,event_subt,...,drv_ti_mat,from,obj_hgt,cr_slp_d,routeid,drv_cs,obj_type,onst,tost,geometry
0,36.08965518,None,S,892,None,-86.62350564,None,0,None,None,...,None,Bell Rd,None,None,8600,None,None,Smith Springs Rd,Rural Hill Rd,POINT (-86.62351 36.08966)
1,36.16262,None,W,390.4704334,None,-86.81184584,None,0,0,FLUSH,...,None,Morena St,None,None,93,0,BIKE FRIENDLY,24th Ave N,Herman St,POINT (-86.81185 36.16262)
2,36.16528346,None,N,161,None,-86.81559108,None,0,None,POINT,...,None,26th Ave N,None,None,26,None,HC-L,Albion St,25th Ave N,POINT (-86.81559 36.16528)
3,36.16549622,None,S,425,None,-86.81060602,None,-1,None,None,...,GRAVEL,24th Ave N,None,None,36,7.4,RESDRW-MID,Albion St,21st Ave N,POINT (-86.81061 36.16550)
4,36.16658087,"LESS THAN 32""",N,218,None,-86.81150264,None,0,None,None,...,None,24th Ave N,None,None,4,None,VEGPT,Alameda St,21st Ave N,POINT (-86.81150 36.16658)


In [4]:
type(sidewalks)

geopandas.geodataframe.GeoDataFrame

In [5]:
sidewalks_by_zip = gpd.sjoin(sidewalks, zipcodes, predicate = 'within')

In [6]:
sidewalks_by_zip.head

<bound method NDFrame.head of            evnt_lat        obj_pot side      measure curbtype      evnt_lon  \
0       36.08965518           None    S          892     None  -86.62350564   
1          36.16262           None    W  390.4704334     None  -86.81184584   
2       36.16528346           None    N          161     None  -86.81559108   
3       36.16549622           None    S          425     None  -86.81060602   
4       36.16658087  LESS THAN 32"    N          218     None  -86.81150264   
...             ...            ...  ...          ...      ...           ...   
188339  36.23846733  LESS THAN 32"    N          352     None  -86.79980543   
188340   36.0674462           None    S            6    32_IN  -86.63103782   
188341  36.00997035           None    N           83    32_IN  -86.68730975   
188342  36.23235414           None    S          646     None  -86.74490148   
188343  36.24333772           None    S         2751     None  -86.75800146   

        sw_mat_ty drv

In [7]:
sidewalks_by_zip['zip'].value_counts()

zip
37208    18177
37203    16853
37206    15045
37212    14601
37209    12174
37013    11389
37207    11248
37210    10696
37211     9566
37138     8194
37214     7410
37076     6761
37204     6675
37221     5299
37115     5145
37216     5102
37205     4243
37217     3126
37215     2862
37027     2628
37201     2065
37218     2030
37213     1980
37219     1117
37228      960
37189      885
37072      518
37135      502
37220      500
37080      201
37232      117
37122      105
37240       81
37086       56
Name: count, dtype: int64

In [8]:
sidewalks_in_37080 = sidewalks_by_zip.loc[sidewalks_by_zip['zip'] == '37080']
sidewalks_in_37080.shape

(201, 31)

In [9]:
polygon37080 = zipcodes.loc[zipcodes['zip'] == '37080']
polygon37080.shape

(2, 6)

In [10]:
polygon37080.geometry.centroid

C:\Users\adaly\AppData\Local\Temp\ipykernel_13288\1019235033.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygon37080.geometry.centroid


6     POINT (-86.86263 36.38001)
20    POINT (-86.89487 36.32034)
dtype: geometry

In [12]:
center = polygon37080.geometry.centroid[20]
print(center)

POINT (-86.89487445940443 36.32034108625658)


C:\Users\adaly\AppData\Local\Temp\ipykernel_13288\2453602238.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = polygon37080.geometry.centroid[20]


In [13]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

[36.32034108625658, -86.89487445940443]


In [14]:
map_37080 = folium.Map(location =  area_center, zoom_start = 12)
map_37080

In [15]:
i = 0
for row_index, row_values in sidewalks_in_37080.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

index is 95169
values are:
 
evnt_lat                               36.31949725
obj_pot                                       None
side                                             W
measure                                        580
curbtype                                      None
evnt_lon                              -86.86698316
sw_mat_ty                                     None
drv_ld                                           0
cr_slp                                        None
event_subt                                   FLUSH
drv_cs_d                                      None
repaired                                        NA
fur_zn_w                                         0
sw_width                                         0
evnt_type                              STORMGRATES
drv_ti_mat                                    None
from                               Eatons Creek Rd
obj_hgt                                       None
cr_slp_d                                      None
ro

In [17]:
#draw zip code area: 37208
map_37080 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37080).add_to(map_37080)

#iterate through sidewalks_in_37208 to create locations and markers 
#for each sidewalk

for row_index, row_values in sidewalks_in_37080.iterrows():
    loc = [row_values['evnt_lat'], row_values['evnt_lon']]
    pop = str(row_values['routeid'])
    icon=folium.Icon(color="orange",icon="fa-brands fa-accessible-icon", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_37080)
map_37080.save('../maps/map37080.html')

#display map
map_37080